## Driver

Drivers are virtualized devices through which applications can interact and initiate I/O operations on the target device.
In GPOS, drivers are part of kernel space and applications are part of user space

A Driver executes in the context of the application.

## Driver Code

It is divided into two parts.

#### Kernel Specific code or User Interaction Code: 
1. Application interfaces with this code to initiate any operation.
2. Therefore Windows, Linux will have different kernel specific code.
3. Writing kernel specific involves kernel programming, kernel testing.

#### Hardware Specific code: 
1. This code performs the operation on the target device.
2. This code will be completely hardware specific like device/bus specific.

## User Interaction Code can be written in three ways:

These are the three different ways of allowing the application to interface with the driver.
1. Character Driver Approach
2. Block Driver Approach
3. Network Driver Approach


## Char Driver Implementation
As a system call vs As a device node file

## Steps to implement and test char driver

1. Create a Device Node File -- mknode
2. Implement the driver operations and register with VFS layer. Now basically each driver is a file system.
3. Insert the driver using a kernel module -- insmod
4. Write an application to test the char driver

### How to create a Device Node File

1. $mknod /location/device_file_name type_of_file_flag 
2. usually location /dev
3. type_of_file: 'c' for char driver, 'b' for block driver
4. Each driver will have a device_file and each device_file is identified by a unique id which formed by combining two different numbers referred as major and minor number. 
5. Internally device id is represented by a field in the inode structure for this particular device file. Each file has an inode and in the inode of this particular device file there is a field called device id of type dev_t.
6. dev_t should be initialised by a device id value which is unique across all device files

#### How to obtain unique major number or device_id?
    1. cat /proc/devices -> Gives you a list a devices in use and their associated
       device_id. Select a number that isn't currently in use.

#### Example
mknod /dev/karthi_device c 300 3
1. device_file_name: karthi_device
2. location: /dev
3. type: character driver
4. major_number: 300
5. minor_number: 3
6. unique_id: 300 3

##### Check
ls -l /dev/karthi_device --- should show all the properties of the device file "karthi_device"


### Skeleton of driver user interaction code

1. #include file1, file2, .... fileN
2. #define MAJORNO MINORNO DEVICENAME
3. Allocate device buffer from kernel space based on the device support for things like DMA
4. Define routines like char_dev_open(arg1, arg2), char_dev_release(arg1, arg2), char_dev_close(arg1, arg2), char_dev_read(arg1, arg2), char_dev_write(arg1, arg2)
        a.arg1: struct inode* inode
        b.arg2: struct file* file
5. Register the addresses of these functions with VFS
        

#### Note
We want VFS to direct all the request made by any user application to karthi_device through common API calls be directed to karthi_device file. For this to happen karthi_device has to be registered with VFS, since VFS is responsible for switching application requests to driver, we need to register addresses of the functions with VFS

### How to register with VFS layer
1. VFS provides a structure of function pointers with the name "file_operations" structure.
2. file_operations structure is defined at "vi /usr/src/current_linux_version/include/linux/fs.h" as struct file_operations
3. In struct file_operations you can find all the operations supported on a file, in our device_file we provide our own implemenatation for operations supported by creating an instance of type file_operations and assigining the pointers to the address of our functions.
4. Bricreate the instance of file_operations and initialized the pointers of file_operations to the operations of our driver. 
        Example: struct file_operations my_instance = {
        .open = char_dev_open;
        .release = char_dev_release;
        .close = char_dev_close;
        .read = char_dev_read;
        .write = char_dev_write;
    };
4. drivers should register their functions with VFS as instances of type file_operations

### An application to test the char driver